In [1]:
try:
    import peft
    LOCAL = True
    MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
    from functions import create_next_model_folder, NoRepeatTokenLogitsProcessor,DigitsOnlyLogitsProcessor,naive_parse
except:
    LOCAL = False
    MODEL_PATH = "/kaggle/input/deepseek-math"
    from functions_math import create_next_model_folder, NoRepeatTokenLogitsProcessor,DigitsOnlyLogitsProcessor,naive_parse
    import gc


import torch
if not LOCAL:torch.backends.cuda.enable_mem_efficient_sdp(False)
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    torch_dtype="auto",
    trust_remote_code = True,
    use_flash_attention_2=LOCAL,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
if LOCAL:
    import json
    with open('../Data/AMC/aime_normal.json', 'r') as file:
        data = json.load(file)
    # to have consistent format as in Kaggle
    data = pd.DataFrame(data)
    data.rename(columns={'question': 'problem'}, inplace=True)
else:
    data = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/test.csv')
    if len(data) < 5:
        data = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
        PRIVATE = False
    else:
        PRIVATE = True

In [3]:
# no_repeat_processor,digits_processor -> rescore parse
# data = pd.read_csv("../llmOutputs/model1"+'.csv')
# data['final_answer'] = data.y
# data = data[data.yhat == 'parsing error']
# data.shape -> (102, 5)

In [4]:
def gen_prompt(problem):
    
    return f"""
### Instruction:\n{problem}\n\n
### Response: Let's think step by step. The final response should be a single number in the last line of your response.
"""

In [5]:
outs = []
no_repeat_processor = [NoRepeatTokenLogitsProcessor()]
digits_processor = [DigitsOnlyLogitsProcessor(tokenizer)]
token2answer = tokenizer.encode("\nthe answer is:", return_tensors="pt",add_special_tokens=False).to('cuda')
for index, row in data.iterrows():
    problem = row['problem']
    query_prompt = gen_prompt(problem)
    
    messages = [
        {
            "role": "user",
            "content": query_prompt
        }
    ]

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    with torch.no_grad():
        encoded_output = model.generate(inputs, max_new_tokens=1500, do_sample=False, pad_token_id=tokenizer.eos_token_id,\
                                        logits_processor=no_repeat_processor)

    decoded_output = tokenizer.decode(encoded_output[0], skip_special_tokens=True).replace(query_prompt, '')
    try:
        answer = decoded_output.split('\n')[-1]
        answer = naive_parse(answer) % 1000
    except:
        # parsing error -> use llm to extract answer
        inputs = torch.cat([encoded_output,token2answer],1)
        with torch.no_grad():
            encoded_output = model.generate(inputs, max_new_tokens=3, do_sample=False, pad_token_id=tokenizer.eos_token_id,\
                                        logits_processor=digits_processor)
        decoded_output = tokenizer.decode(encoded_output[0], skip_special_tokens=True)
        print(decoded_output)
        try:
            answer = naive_parse(decoded_output.split('\n')[-1]) % 1000
        except:
            answer = 'parsing error'
    if LOCAL:
        outs.append((problem,decoded_output,answer,int(row['final_answer'][0])))
    else:
        outs.append(37 if answer == 'parsing error' else answer)
        torch.cuda.empty_cache()
        gc.collect()
        if not PRIVATE:
            print(decoded_output)
            print(f'\nanswer is {answer}')

User: 
### Instruction:
Positive real numbers <math>b \not= 1</math> and <math>n</math> satisfy the equations <cmath>\sqrt{\log_b n} = \log_b \sqrt{n} \qquad \text{and} \qquad b \cdot \log_b n = \log_b (bn).</cmath> The value of <math>n</math> is <math>\frac{j}{k},</math> where <math>j</math> and <math>k</math> are relatively prime positive integers. Find <math>j+k.</math>


### Response: Let's think step by step. The final response should be a single number in the last line of your response.


The first equation is $\sqrt{\log_b n} = \log_b \sqrt{n}$.
Squaring both sides, we get $\log_b n = \log_b n$, which is true.
So the first equation is satisfied for all $n > 0$.
The second equation is $b \cdot \log_b n = \log_b (bn)$.
Using the property of logarithms, we can rewrite this as $\log_b n^b = \log_b (bn)$.
This simplifies to $\log_b n^b = \log_b n + \log_b b = \log_b n + 1$.
So we have $\log_b n^b = \log_b n + 1$.
This is equivalent to $n^b = b \cdot n$, which simplifies to $n^{b-1} =

In [ ]:
if LOCAL:
    outs_df = pd.DataFrame(outs,columns=['problem','output','yhat','y'])
    print(sum(outs_df.yhat == outs_df.y)/outs_df.shape[0])
    out_path = create_next_model_folder('../llmOutputs')
    print(out_path) # ../llmOutputs/model1
    outs_df.to_csv(out_path+'.csv', header=True, index=False)
else:
    if not PRIVATE:
        answers = data.answer.tolist()
        correct = sum([y==yhat for y,yhat in zip(answers,outs)])
        print(f'{correct} correct answers')    
    data['answer'] = outs
    data[['id','answer']].to_csv("submission.csv", header=True, index=False)

0.0


TypeError: create_next_model_folder() takes from 0 to 1 positional arguments but 2 were given